In [2]:
import pandas as pd
# import numpy as np
import os
import sys

dir_path = '/home/pamya/Python/ML_Projects/Pride-of-Sahyadri'
sys.path.append(dir_path)
os.chdir(dir_path)

In [ ]:
df = pd.read_csv('data/maharashtra-forts.csv')
df.sample(3)

In [ ]:
df.info()

## Option 1 — Import Routers & Call Functions Directly

In [ ]:
from src.core.data_loader import load_forts
from src.api.routers.forts import list_forts, get_fort

In [ ]:
# Test data loading
df = load_forts()
df.head()

In [ ]:
result = list_forts(limit=5, q=None, district=None)
result

In [ ]:
result[0]

In [ ]:
result = list_forts(limit=10, q="shivaji", district=None)
result

In [ ]:
from src.api.routers.clustering import get_clusters, predict_cluster

get_clusters()

In [ ]:
predict_cluster(lat=18.52, lon=73.85)

In [ ]:
from src.core.rag_engine import RAGEngine

rag = RAGEngine()
rag.load_data(df=df)
rag.build_index()

In [ ]:
result = rag.query("sea fort with strong historical importance", k=3)
result

In [ ]:
from src.core.recommender import recommend_by_proximity, recommend_similar

In [ ]:
df0 = recommend_by_proximity(df, lat=18.52, lon=73.85, k=10)
df0

In [ ]:
df2 = recommend_similar(df, fort_id=1, k=5)
df2.head(5)

## Option 2 — Use FastAPI TestClient

In [ ]:
from fastapi.testclient import TestClient
from src.api.main import app

client = TestClient(app=app)

In [ ]:
r = client.get("/forts")
print(r.status_code)
r.json()[:3]

In [ ]:
client.get(f"/forts/{301}").json()

In [ ]:
client.get("/forts", params={"q": "Sindhudurg"}).json()[:3]

In [5]:
client.get(
    "/search/semantic_search",
    params={"q": "sea forts built by shivaji"}
).json()

Loading Qwen/Qwen2-1.5B-Instruct on cpu...


{'error': 'Tokenizer class Qwen2Tokenizer does not exist or is not currently imported.'}

In [ ]:
tests = {
    "forts_list": client.get("/forts"),
    "search": client.get("/forts", params={"q":"hill"}),
    "single_fort": client.get("/forts/1"),
    "cluster_summary": client.get("/clusters"),
    "cluster_predict": client.get("/clusters/predict", params={"lat":18.5,"lon":73.8}),
    "nearby": client.get("/recommend/nearby", params={"lat":18.5,"lon":73.8,"k":3}),
    "similar": client.get("/recommend/similar/10", params={"k":3}),
    "rag_search": client.get("/search/semantic_search", params={"q":"sea fort in Sindhudurg district"})
}

for name, result in tests.items():
    print(f"\n--- {name.upper()} ---")
    print("Status:", result.status_code)
    print("Sample:", result.json()[:2] if isinstance(result.json(), list) else result.json())


In [ ]:
import requests
import pandas as pd

BASE = "http://localhost:8000"

def show(x):
    """Pretty print JSON or DataFrame intelligently."""
    if isinstance(x, list):
        if len(x) == 0:
            print("[] (empty list)")
        else:
            display(pd.DataFrame(x).head())
    elif isinstance(x, dict):
        print(x)
    else:
        print(x)


In [ ]:
try:
    r = requests.get(f"{BASE}/forts")
    print("STATUS:", r.status_code)
except Exception as e:
    print("❌ Cannot reach backend:", e)

In [ ]:
forts = requests.get(f"{BASE}/forts").json()
show(forts)

In [ ]:
clusters = requests.get(f"{BASE}/data")
print("Cluster Counts:")
print(clusters)

In [3]:
from fastapi.testclient import TestClient
from src.api.main import app

client = TestClient(app=app)

RAGEngine: Corpus created with 346 entries.
RAGEngine: Loading cached embeddings...
RAGEngine: Embeddings loaded from cache.


In [ ]:
client.get("/clusters").json()

In [ ]:
clustered = client.get("/clusters/data").json()

In [ ]:
clustered[0]

In [ ]:
client.get("/recommend/nearby", params={"lat":18.5,"lon":73.8,"k":3}).json()


In [ ]:
client.get("/recommend/similar/2", params={"k":3}).json()

In [ ]:
clustered[25]['fort_id']

In [ ]:
fort_id = clustered[25]['fort_id']  # or if fort_id exists: clustered[0]["fort_id"]

similar = client.get(f"/recommend/similar/{fort_id}", params={"k": 3}).json()

similar

In [4]:
client.get("/search/semantic_search", params={"q":"Which forts are difficult treks?"}).json()

Loading Qwen/Qwen2-1.5B-Instruct on cpu...


/home/pamya/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


{'error': 'Tokenizer class Qwen2Tokenizer does not exist or is not currently imported.'}

In [ ]:
from src.core.cluster_engine import ClusterEngine

cluster_engine = ClusterEngine()
cluster_engine.build_clusters()

In [ ]:
clustered_df = cluster_engine.get_clustered_data()

In [ ]:
clustered_df.iloc[0]